In [ ]:
import aliases

In [ ]:
from everest.utilities import classtools as _classtools

In [ ]:
@_classtools.MROClassable
class A:
    @_classtools.MROClass
    class B:
        ...

class C(A):
    class B:
        ...

In [ ]:
@_classtools.MROClassable
@_classtools.Reloadable
class A:
    @_classtools.Overclass
    class B:
        ...
        @_classtools.Overclass
        class C:
            ...

In [ ]:
myobj = A.B.C()

In [ ]:
A.B.C.get_classpath()

In [ ]:
import pickle
pipickle.dumps(myobj)

In [ ]:
A.B.overclassmrobases

In [ ]:
dir(A.B)

In [ ]:
tuple(_classtools.mroclasses.yield_mroclassnames(A.B))

In [ ]:
issubclass(A.B, _classtools.mroclasses.Overclass)

In [ ]:
_classtools.m

In [ ]:
myset = set()

In [ ]:
myset.update(iter('aabcd'))

In [ ]:
myset

In [ ]:
A.B.mroclassowner

In [ ]:
A.B.__init_subclass__()

In [ ]:
A.B.C.__mro__

In [ ]:
A.B.__init_subclass__()

In [ ]:
A.B.C.__mro__

In [ ]:
A.__init_subclass__()

In [ ]:
###############################################################################
''''''
###############################################################################


import functools as _functools
from abc import ABCMeta as _ABCMeta
import itertools as _itertools
from collections import abc as _collabc

from everest.utilities.misc import (
    TypeMap as _TypeMap,
    )
from everest.utilities import classtools as _classtools


class ChoraMeta(_ABCMeta):

    def __init__(cls, /, *args, **kwargs):
        super().__init__(*args, **kwargs)
        cls._cls_extra_init_()

    def decorate(cls, ACls, /):
        old_getitem = ACls.__getitem__
        old_init = ACls.__init__
        def chora_extra_init(self, *args, **kwargs):
            chora = self.chora = cls()
            self._choraget = _functools.partial(
                chora.__getitem__,
                retrieve = _functools.partial(old_getitem, self)
                )
            old_init(self, *args, **kwargs)
        ACls.__init__ = chora_extra_init
        def __getitem__(self, incisor):
            return self._choraget(incisor)
        ACls.__getitem__ = __getitem__
        return ACls


_IncMethsLike = _collabc.Iterator[tuple[type, _collabc.Callable]]


@_classtools.MROClassable
@_classtools.Diskable
class ChoraBase(metaclass=ChoraMeta):

    @_classtools.MROClass
    @_classtools.Diskable
    class Element:

        def __init__(self, /, incisor):
            self.incisor = incisor
            self.register_argskwargs(incisor)

        def __call__(self, /, retrieve):
            return retrieve(self.incisor)

    @classmethod
    def incision_methods(cls, /) -> _IncMethsLike:
        '''Returns acceptable incisor types and their associated getmeths.'''
        return iter(())

    @classmethod
    def priority_incision_methods(cls, /) -> _IncMethsLike:
        '''Returns like `.incision_methods` but takes priority.'''
        return iter(())

    @classmethod
    def _get_incmeths(cls, /) -> _TypeMap:
        return _TypeMap(
            _itertools.chain(
                cls.priority_incision_methods(),
                cls.incision_methods()
                ),
            )

    @classmethod
    def _cls_extra_init_(cls, /):
        cls.incmeths = cls._get_incmeths()
        def __getitem__(self, incisor, /, *, retrieve=cls.Element):
            try:
                meth = self.incmeths[type(incisor)]
            except KeyError as exc:
                raise TypeError from exc
            return meth(self, incisor, retrieve=retrieve)
        cls.__getitem__ = __getitem__

    def incise_strict(self, incisor, /, *, retrieve):
        return retrieve(incisor)

class Chora(ChoraBase):

    @_classtools.Overclass
    class Incision(ChoraBase):

        def __init__(self, source, /):
            self.source = source
            self.register_argskwargs(source)

    def __init__(self, /, *, criterion):
        self.register_argskwargs(criterion=criterion)
        if isinstance(type(criterion), type):
            criterionclass = criterion
            def criterion(arg):
                return isinstance(arg, criterionclass)
        elif not callable(criterion):
            raise TypeError(criterion)
        self.criterion = criterion

    def __contains__(self, arg, /):
        return self.criterion(arg)


###############################################################################
###############################################################################

In [ ]:
class MyChora(Chora):

    def __init__(self):
        super().__init__(criterion=int)

    @classmethod
    def incision_methods(cls):
        yield int, cls.incise_strict
        yield from super().incision_methods()

@MyChora.decorate
class MyClass:

    def __init__(self, *args, **kwargs):
        self.contents = list(*args, **kwargs)

    def __getitem__(self, index):
        return self.contents[index]

myobj = MyClass('abcdefg')
assert myobj[0] == 'a'

In [ ]:
###############################################################################
''''''
###############################################################################


import itertools as _itertools

from everest import utilities as _utilities

_Chora = Chora


class Sliceable(_Chora):

    @classmethod
    def _cls_extra_init_(cls, /):
        super()._cls_extra_init_()
        slcmeths = cls.slcmeths = dict(zip(
            _itertools.product(*_itertools.repeat((True, False), 3)),
            _itertools.repeat(cls.incise_slice_bad),
            ))
        slcmeths.update(reversed(tuple(cls.slice_methods())))

    @classmethod
    def slice_methods(cls, /):
        return iter(())

    def incise_slice(self, slc, /, *, retrieve):
        return self.incise_slyce(
            _utilities.misc.slyce(slc),
            retrieve=retrieve,
            )

    def incise_slyce(self, slc, /, *, retrieve):
        return self.slcmeths[slc.hasargs](self, *slc, retrieve=retrieve)

    def incise_slice_bad(self, /, *slcargs, retrieve):
        raise ValueError(" ".join((
            f"Object of type {type(self)}",
            "cannot be sliced with slice of",
            "start={0}, stop={1}, step={2}".format(*slcargs),
            )))

    @classmethod
    def incision_methods(cls, /):
        yield slice, cls.incise_slice
        yield _utilities.misc.Slyce, cls.incise_slyce
        yield from super().incision_methods()


###############################################################################
###############################################################################

In [ ]:
###############################################################################
''''''
###############################################################################


import operator as _operator

from everest.utilities import classtools as _classtools

_Sliceable = Sliceable


class Orderable(_Sliceable):

    class Incision:

        @_classtools.Overclass
        class Delimited:

            def __init__(self, *args, lbnd, ubnd, **kwargs):
                self.lbnd, self.ubnd = lbnd, ubnd
                self.register_argskwargs(lbnd=lbnd, ubnd=ubnd)
                super().__init__(*args, **kwargs)

            def incise_delimit(self, start, stop, _, /, *, retrieve):
                raise Exception("Cannot delimit already delimited.")

    def __init__(self, /, *, comparator=_operator.gt, **kwargs):
        self.comparator = comparator
        self.register_argskwargs(comparator=comparator)
        super().__init__(**kwargs)

    def incise_delimit(self, start, stop, _, /, *, retrieve):
        return self.Incision.Delimited(self, lbnd=start, ubnd=stop)

    @classmethod
    def slice_methods(cls, /):
        for comb in ((True, True), (True, False), (False, True)):
            yield (*comb, False), cls.incise_delimit
        yield from super().slice_methods()


class Enumerable(Orderable):

    class Incision:

        class Delimited:

            ...


###############################################################################
###############################################################################

In [ ]:
issubclass(Orderable.Incision, _classtools.mroclasses.MROClassable)

In [ ]:
Orderable.Incision.Delimited.__mro__

In [ ]:
class MyChora(Orderable):

    def __init__(self):
        self.freeze_argskwargs()
        super().__init__(criterion=int)

    @classmethod
    def incision_methods(cls):
        yield int, cls.incise_strict
        yield from super().incision_methods()


@MyChora.decorate
class MyClass:

    def __init__(self, *args, **kwargs):
        self.contents = list(*args, **kwargs)

    def __getitem__(self, index):
        return self.contents[index]


import string
myobj = MyClass(string.ascii_lowercase)
assert myobj[0] == 'a'

In [ ]:
myinc = myobj[10:20]

In [ ]:
myinc[1:2]

In [ ]:
list(myinc.source.get_redtup())

In [ ]:
import pickle
dumped = pickle.dumps(myinc)
loaded = pickle.loads(dumped)

In [ ]:
loaded